In [ ]:
import os
import datetime

import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
cytosim_fld = '/media/lab/Data/Fabio/Dev/Python-actomyosin-furrows/cytosim/'
df = (pd.read_csv(cytosim_fld + 'actomyosin_sweep/actomyosin_treadmill.csv', skipinitialspace=True)
      .drop(columns='n')
     )
df['run'] = df['file'].apply(lambda r: 'run' + r.split('.')[0][-4:])

df

In [ ]:
import imageio

# report values in kilo units
df['N_fil'] /= 1000.0 
df['N_crs'] /= 1000.0 
df['N_myo'] /= 1000.0 

condition0 = sorted(df['N_fil'].unique())
condition1 = sorted(df['N_crs'].unique())
condition2 = sorted(df['N_myo'].unique())
# print([condition0, condition1, condition2])

# build dataframe of last image given condition coordinates
for c0 in condition0:
    T = []
    for c1 in condition1:
        for c2 in condition2:
            r = df.query('N_fil==@c0 and N_crs==@c1 and N_myo==@c2')
            im_path = f"{cytosim_fld}actomyosin_sweep/treadmill/{r['run'].values[0]}/image1000.png"
            T.append({'crs':c1,
                      'myo':c2,
                      'picture':imageio.imread(im_path),
                      })
    ph = pd.DataFrame(T)
    
    grid = sns.FacetGrid(ph, row='crs', col='myo', height=1, aspect=1, margin_titles=True, despine=True)
    grid.map(lambda x, **kwargs : (plt.imshow(x.values[0]),plt.grid(False)), 'picture')
    grid.set_xlabels("")
    grid.set(xticks=[], yticks=[])
    grid.set_titles(row_template="{row_var} {row_name}k", col_template="{col_var} {col_name}k")
    grid.fig.suptitle(f"{int(c0*1000)} filaments")
    grid.tight_layout()
    grid.fig.subplots_adjust(wspace=0, hspace=0)
    grid.savefig(f"f{int(c0*1000):05d}-c{int(c1*1000):05d}-m{int(c2*1000):05d}.pdf")
